In [2]:
from google.colab import files
uploaded = files.upload()


Saving archive.zip to archive.zip


In [4]:
import zipfile
import io

with zipfile.ZipFile(io.BytesIO(uploaded['archive.zip']), 'r') as zip_ref:
    zip_ref.extractall("dataset")


In [5]:
import os

dataset_path = "dataset"  # or your folder name
for root, dirs, files in os.walk(dataset_path):
    print(root, len(files), "files")


dataset 0 files
dataset/data 0 files
dataset/data/with_mask 3725 files
dataset/data/without_mask 3828 files


Found 6042 images belonging to 2 classes.
Found 1511 images belonging to 2 classes.


In [19]:
import os

# Change this if your dataset extracted somewhere else
dataset_path = "data"  # or whatever `print(path)` from kagglehub gave

for root, dirs, files in os.walk(dataset_path):
    print(root, len(files), "files")


data 0 files
data/train 0 files
data/train/with_mask 2980 files
data/train/without_mask 3062 files
data/val 0 files
data/val/with_mask 745 files
data/val/without_mask 766 files


In [11]:
!pip install split-folders

In [26]:
import splitfolders

input_folder = "dataset/data"  # the folder with all class folders
splitfolders.ratio(input_folder, output="data", seed=42, ratio=(0.8, 0.2))

Copying files: 7553 files [00:01, 4810.43 files/s]


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "val")

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)


Found 6042 images belonging to 2 classes.
Found 1511 images belonging to 2 classes.


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Change 3 to number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
history = model.fit(train_generator, epochs=10, validation_data=test_generator)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 55/189 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.6266 - loss: 0.9170

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 21s 86ms/step - accuracy: 0.7494 - loss: 0.6062 - val_accuracy: 0.9100 - val_loss: 0.2293
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.8885 - loss: 0.2560 - val_accuracy: 0.9047 - val_loss: 0.2642
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.9246 - loss: 0.1849 - val_accuracy: 0.9351 - val_loss: 0.2071
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - accuracy: 0.9506 - loss: 0.1293 - val_accuracy: 0.9365 - val_loss: 0.2094
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 21s 63ms/step - accuracy: 0.9618 - loss: 0.1022 - val_accuracy: 0.9385 - val_loss: 0.2128
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.9738 - loss: 0.0725 - val_accuracy: 0.9351 - val_loss: 0.2191
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 13s 66ms/step - accuracy: 0.9822 - loss: 0.0545 - val_accuracy: 0.9279 - val_loss: 0.2574
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.9844 - loss: 0.0487 - val_accurac

In [30]:
model.save("face_mask_model.h5")


In [43]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Load the trained model
model = tf.keras.models.load_model("face_mask_model.h5")

# Class labels
classes = ["Mask", "No Mask"]

# Path to your single test image
test_image_path = "/test_images.jpg" # Replace with the actual path to your image file

# Load the image
img = cv2.imread(test_image_path)

if img is None:
    print(f"Error: Could not load image from {test_image_path}")
else:
    # Preprocess the image
    img_resized = cv2.resize(img, (128, 128)) # Resize to the model's expected input size
    img_array = tf.keras.preprocessing.image.img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Make prediction
    prediction = model.predict(img_array)
    predicted_class = classes[np.argmax(prediction)]

    print(f"{os.path.basename(test_image_path)} → {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 750ms/step
test_images.jpg → Mask
